In [ ]:
import http.client

conn = http.client.HTTPSConnection("api.askedith.ai")

payload = '{"question": "What is the average sales?", "data_source_id": "1234567890", "tables": ["table_1", "table_2"], "datasets": ["dataset_1", "dataset_2"]}'

headers = { 'Authorization': "" }

conn.request("POST", "/api/query", payload, headers)

res = conn.getresponse()

data = res.read()

print(data.decode("utf-8"))

In [10]:
import http.client

conn = http.client.HTTPSConnection("api.askedith.ai")

payload = r'{"question": "What is the average sales?", "files": { "train.csv": "column1,column2,column3"}, "auto_run": false}'

headers = { 'Authorization': "Bearer " }

conn.request("POST", "/api/query/local", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"error": "Please upgrade your subscription or contact support@askedith.ai to get access to the API!"}


In [8]:
import http.client

conn = http.client.HTTPSConnection("api.askedith.ai")

headers = { 'Authorization': "Bearer " }

conn.request("GET", "/api/data-sources", "", headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

Please upgrade your subscription or contact support@askedith.ai to get access to the API!


In [2]:
import pandas as pd
df  = pd.read_csv("train.csv")

# print column names of df
print(df.columns)

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='object')


In [ ]:
with open('generated_queries_all.tsv', 'w') as fOut:
    for start_idx in tqdm(range(0, len(paragraphs), batch_size)):
        sub_paragraphs = paragraphs[start_idx:start_idx+batch_size]
        inputs = tokenizer.prepare_seq2seq_batch(sub_paragraphs, max_length=max_length_paragraph, truncation=True, return_tensors='pt').to(device)
        outputs = model.generate(
            **inputs,
            max_length=max_length_query,
            do_sample=True,
            top_p=0.95,
            num_return_sequences=num_queries)

        for idx, out in enumerate(outputs):
            query = tokenizer.decode(out, skip_special_tokens=True)
            query = _removeNonAscii(query)
            para = sub_paragraphs[int(idx/num_queries)]
            para = _removeNonAscii(para)
            fOut.write("{}\t{}\n".format(query.replace("\t", " ").strip(), para.replace("\t", " ").strip()))